In [8]:
import pandas as pd
import os
import shutil
import zipfile
import cv2
import uuid
import random

In [9]:
def count_files(folder):
    try:
        file_count = len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))])
        print(f"Total files in '{folder}' folder: {file_count}")
    except Exception as e:
        print(f"Error occurred: {e}")

In [10]:
VID_FOLDER = 'vids'
PIC_FOLDER = 'randomPics'
ORIGINAL_PIC_FOLDER = 'randomPicOriginals'
TABLE_NAME = 'picsToVids'

count_files(VID_FOLDER)

Total files in 'vids' folder: 324


In [11]:
NUMBER_VIDEOS = 324
NUMBER_FRAMES = 250
SQUARE_SIDE_SIZE = 500

In [12]:
def extract_random_frames(video):
    
    # Create the output folders if doesn't exist
    if not os.path.exists(PIC_FOLDER):
        os.makedirs(PIC_FOLDER)
    
    if not os.path.exists(ORIGINAL_PIC_FOLDER):
        os.makedirs(ORIGINAL_PIC_FOLDER)
    
    # Open the video file
    full_video_path = os.path.join(VID_FOLDER, video)
    cap = cv2.VideoCapture(full_video_path)
    
    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cut_percent = int(total_frames*0.12)
    
    # Generate N unique random frame numbers
    random_frames = random.sample(range(cut_percent,total_frames-cut_percent), min(NUMBER_FRAMES, total_frames-5*cut_percent))
    
    data = []
    for frame_number in random_frames:
        # Set the current frame position of the video file
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        
        # Read the frame
        success, frame = cap.read()
        
        # If reading was successful, save the frame as an image
        if success:
            
            # save the uncropped frame
            filename = f"{uuid.uuid4()}.jpg"
            cv2.imwrite(os.path.join(ORIGINAL_PIC_FOLDER, filename), frame)
            
            # crop
            height, width = frame.shape[:2]
            center_x, center_y = width // 2, height // 2
            x_start = center_x - SQUARE_SIDE_SIZE // 2
            x_end = center_x + SQUARE_SIDE_SIZE // 2
            y_start = center_y - SQUARE_SIDE_SIZE // 2
            y_end = center_y + SQUARE_SIDE_SIZE // 2
            cropped_frame = frame[y_start:y_end , x_start:x_end]

            cv2.imwrite(os.path.join(PIC_FOLDER, filename), cropped_frame)

            # add to the table
            data.append({'picname': filename, 'videoname': video})
        else:
            print(f"{video} : Failed to extract frame {frame_number}")

    # Release the video capture object
    cap.release()
    # print(f"{video} : Done")

    return data 

In [13]:
counter = 0
data = []

try:
    for video in os.listdir(VID_FOLDER):

        if counter >= NUMBER_VIDEOS:
            break

        data += extract_random_frames(video)
        counter += 1
finally:
    TABLE = pd.DataFrame(data)
    TABLE.to_csv(TABLE_NAME + '.csv', index=False)


In [14]:
count_files(PIC_FOLDER)
count_files(ORIGINAL_PIC_FOLDER)

Total files in 'randomPics' folder: 80371
Total files in 'randomPicOriginals' folder: 80371


In [1]:
def split_into_folders(source_folder, target_root, pics_per_folder=50):
    if not os.path.exists(target_root):
        os.makedirs(target_root)
    
    files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    total_files = len(files)
    folder_count = 0
    
    for i in range(0, total_files, pics_per_folder):
        folder_name = os.path.join(target_root, f"patch_{folder_count}")
        os.makedirs(folder_name, exist_ok=True)
        
        for file in files[i:i+pics_per_folder]:
            shutil.move(os.path.join(source_folder, file), folder_name)
        
        folder_count += 1


split_into_folders('randomPics', 'splitPics', 50)


In [2]:
def compress_folders(root_folder):
    compressed_folder = root_folder + 'Compressed'
    if not os.path.exists(compressed_folder):
        os.makedirs(compressed_folder)
    
    folders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]
    for folder in folders:
        folder_path = os.path.join(root_folder, folder)
        zip_file_path = os.path.join(compressed_folder, f"{folder}.zip")
        with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(folder_path, '..')))

compress_folders('splitPics')